#IMPORT ALL LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# LOAD THE DATASET

In [5]:
dataset = pd.read_csv('Tweets.csv')

# DATA PREPROCESSING

In [6]:
dataset.shape

(14640, 15)

In [7]:
dataset.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [8]:
dataset.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [9]:
tweets = dataset.drop(['tweet_id', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],axis=1)

In [10]:
print(tweets.shape)

(14640, 2)


In [11]:
print(tweets.columns)

Index(['airline_sentiment', 'text'], dtype='object')


In [12]:
tweets[:5]

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [13]:
np.unique(tweets['airline_sentiment'])

array(['negative', 'neutral', 'positive'], dtype=object)

#LABELS PREPROCESSING

Handling Categorical Data

In [14]:
labels = tweets['airline_sentiment']
for i in labels:
  if i=='negative':
    i=0
  elif i=='neutral':
    i=1
  else:
    i=2

#TEXT PREPROCESSING

Converting to Lower Case

In [15]:
for i in range(len(tweets['text'])):
  tweets['text'][i] = tweets['text'][i].lower()

Removal of @

In [16]:
def remove_mentions(word):       
    result = re.sub(r"@\S+", "", word)       
    return result

for i in range(len(tweets['text'])):
    tweets['text'][i] = remove_mentions(tweets['text'][i])

Removal of HTML Tag

In [17]:
def remove_hyperlink(word):       
    return re.sub(r"http\S+", "", word)

for i in range(len(tweets['text'])):
  tweets['text'][i] = remove_hyperlink(tweets['text'][i])

Removal of Special Characters

In [18]:
def remove_special_characters(word):       
    result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))    
    return result

for i in range(len(tweets['text'])):
  tweets['text'][i] = remove_special_characters(tweets['text'][i])

Removal of Puntuation

In [19]:
for i in range(len(tweets['text'])):
  tweets['text'][i] = "".join([char.lower() for char in tweets['text'][i] if char not in string.punctuation])

Remove any extra White Spaces

In [20]:
for i in range(len(tweets['text'])):
  tweets['text'][i] = re.sub('\s+', ' ', tweets['text'][i]).strip()

Removal of Stopwords

In [21]:
ENGLISH_STOP_WORDS = set(stopwords.words('english'))
def remove_stop_words(words):
    mylist = words.split(" ")       
    result = " ".join([i for i in mylist if i not in ENGLISH_STOP_WORDS])     
    return result

for i in range(len(tweets['text'])):
  tweets['text'][i] = remove_stop_words(tweets['text'][i])

Removal of Numbers

In [22]:
for i in range(len(tweets['text'])):
  tweets['text'][i] = re.sub(r'\d+', '', tweets['text'][i])

Tokenization

In [23]:
tweet_text_list = []
for tweet in tweets['text']:
    print(word_tokenize(tweet))
    tweet_text_list.append(word_tokenize(tweet))

Streaming output truncated to the last 5000 lines.
['mention', 'fact', 'flight', 'delayed', 'computers']
['can', 'not', 'add', 'frequent', 'flyer', 'number', 'website', 'help']
['get', 'plane', 'wait', 'hour', 'half', 'speak', 'someone', 'reflight', 'booking', 'problems']
['please', 'dont', 'trivialize', 'joke', 'slow', 'day', 'slow', 'experience']
['agent', 'phx', 'customer', 'service', 'desk', 'hour', 'ago', 'line', 'people', 'long']
['sorry', 'doesnt', 'help', 'midnight', 'pst', 'hell', 'still', 'hold', 'minutes', 'customer', 'service', 'line']
['like', 'thought', 'america', 'us', 'airways', 'start', 'operating', 'rules', 'communist', 'russia']
['guys', 'nice', 'give', 'routes', 'jet', 'blue', 'southwest', 'least', 'treat', 'customers', 'properly']
['nope', 'apparently', 'way', 'guys', 'business', 'guess', 'im', 'flying', 'southwest', 'blue']
['keep', 'thinking', 'massive', 'practical', 'joke', 'someone', 'gon', 'na', 'appear', 'thin', 'air', 'take', 'reno', 'tonight']
['theres', 'p

In [24]:
tweet_text_list

[['said'],
 ['plus', 'youve', 'added', 'commercials', 'experience', 'tacky'],
 ['didnt', 'today', 'must', 'mean', 'need', 'take', 'another', 'trip'],
 ['really',
  'aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guests',
  'faces',
  'amp',
  'little',
  'recourse'],
 ['really', 'big', 'bad', 'thing'],
 ['seriously',
  'would',
  'pay',
  'flight',
  'seats',
  'didnt',
  'playing',
  'really',
  'bad',
  'thing',
  'flying',
  'va'],
 ['yes',
  'nearly',
  'every',
  'time',
  'fly',
  'vx',
  '“',
  'ear',
  'worm',
  '”',
  'won',
  '’',
  't',
  'go',
  'away'],
 ['really',
  'missed',
  'prime',
  'opportunity',
  'men',
  'without',
  'hats',
  'parody'],
 ['well', 'didnt…but'],
 ['amazing', 'arrived', 'hour', 'early', 'youre', 'good'],
 ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teens'],
 ['lt', 'pretty', 'graphics', 'much', 'better', 'minimal', 'iconography'],
 ['great',
  'deal',
  'already',
  'thinking',
  'nd',
  'trip',
  'amp',
  'h

Lemmatize or Stemming

In [25]:
stemmer = PorterStemmer()
def stem_words(text):       
    return "".join([stemmer.stem(word) for word in text])

for i in range(len(tweets['text'])):
  tweets['text'][i] = stem_words(tweets['text'][i])

Print first 5 rows of data after pre-processing

In [26]:
tweets['text'][:5]

0                                                 said
1        plus youve added commercials experience tacky
2         didnt today must mean need take another trip
3    really aggressive blast obnoxious entertainmen...
4                                 really big bad thing
Name: text, dtype: object

# VECTORIZATION

Using CountVectorizer

In [27]:
cvector = CountVectorizer(min_df=2,max_features=100000)
cvector.fit(tweets['text'])
tweets_processed = cvector.transform(tweets['text']).toarray()

In [28]:
X1_train, X1_test, y1_train, y1_test = train_test_split(tweets_processed, labels, test_size=0.2, random_state=30)

In [29]:
X1_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
X1_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
y1_train

6431     positive
11383    negative
7714     positive
1610      neutral
7199      neutral
           ...   
13452    negative
500       neutral
12077     neutral
4517     negative
5925      neutral
Name: airline_sentiment, Length: 11712, dtype: object

In [32]:
y1_test

7333     negative
14080    negative
7771      neutral
11114    negative
8068      neutral
           ...   
652       neutral
9053      neutral
198      negative
8779     negative
13951     neutral
Name: airline_sentiment, Length: 2928, dtype: object

Using Tf-idfVectorizer

In [33]:
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

tf_vector = get_feature_vector(np.array(tweets['text']).ravel())
X2 = tf_vector.transform(np.array(tweets['text']).ravel())
y2 = np.array(labels).ravel()

In [34]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=30)

# FITTING THE MODELS

Naive Bayes Model

1. For CountVectorization
2. For Tf-idf

In [35]:
NB_model1 = MultinomialNB()
NB_model1.fit(X1_train, y1_train)

MultinomialNB()

In [36]:
NB_model2 = MultinomialNB()
NB_model2.fit(X2_train, y2_train)

MultinomialNB()

Logistic Regression Model

1. For CountVectorization
2. For Tf-idf

In [37]:
LR_model1 = LogisticRegression(max_iter=1000, verbose=1, n_jobs=-1)
LR_model1.fit(X1_train, y1_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.4min finished


LogisticRegression(max_iter=1000, n_jobs=-1, verbose=1)

In [38]:
LR_model2 = LogisticRegression(max_iter=1000, verbose=3, n_jobs=-1)
LR_model2.fit(X2_train, y2_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.8s finished


LogisticRegression(max_iter=1000, n_jobs=-1, verbose=3)

# EVALUATING THE MODELS

Naive Bayes Model

In [40]:
y1_predict_nb = NB_model1.predict(X1_test)
print(y1_predict_nb)
print(accuracy_score(y1_test, y1_predict_nb))

['negative' 'negative' 'positive' ... 'negative' 'negative' 'negative']
0.7721994535519126


In [41]:
y2_predict_nb = NB_model2.predict(X2_test)
print(y2_predict_nb)
print(accuracy_score(y2_test, y2_predict_nb))

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']
0.6885245901639344


Logistic Regression Model

In [42]:
y1_predict_lr = LR_model1.predict(X1_test)
print(y1_predict_lr)
print(accuracy_score(y1_test, y1_predict_lr))

['negative' 'negative' 'positive' ... 'negative' 'neutral' 'neutral']
0.7885928961748634


In [43]:
y2_predict_lr = LR_model2.predict(X2_test)
print(y2_predict_lr)
print(accuracy_score(y2_test, y2_predict_lr))

['negative' 'negative' 'positive' ... 'negative' 'negative' 'neutral']
0.7824453551912568


In [44]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(LR_model1, open(filename, 'wb'))